importer fichier cvs

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
df = pd.read_csv('data.csv')

Data Cleaning

In [ ]:
to_drop = ["Date"
,"ConcoursOuvertMois"
,"ConcoursOuvertAnnée"
,"PromotionCompétitionSemaine"
,"PromotionCompétitionAnnée"
,"Mois"
,"Trimestre"
,"Année"
,"Jour"
,"Semaine"
,"PromotionIntermédiaire"]
df['PromotionCompétitionSemaine'].fillna(0, inplace=True)
df['Mois-année'] = df['Mois'].astype(str) + '-' + df['Année'].astype(str)

In [ ]:
print(pd.cut(df["Ventes"], bins=[0, 5000, 10000,350000], labels=[0, 1, 2]).value_counts())
df["cat_Ventes"] = pd.cut(df["Ventes"], bins=[0, 5000, 10000,350000], labels=[0, 1, 2])
df["cat_Ventes"] = df["cat_Ventes"].fillna(2)

In [ ]:
df = df.dropna()
df = df.drop(to_drop, axis=1)

num_cols = df._get_numeric_data().columns
# select the categorical columns
cat_cols = df.select_dtypes(include='object').columns

# apply One Hot Encoding on the categorical columns
df = pd.get_dummies(df, columns=cat_cols)

Display Data

In [ ]:
# Regroupement des données par trimestre et promotion, en calculant la moyenne des ventes
grouped_data = df.groupby(['Mois-année', 'PromotionsClassiques'])['Ventes'].mean().reset_index()

# Création des courbes pour chaque niveau de promotion
promotions = grouped_data['PromotionsClassiques'].unique()

for promotion in promotions:
    subset = grouped_data[grouped_data['PromotionsClassiques'] == promotion]
    plt.plot(subset['Mois-année'], subset['Ventes'], label=promotion)

plt.xlabel('Mois-année')
plt.ylabel('Ventes')
plt.title('Relation entre ventes, date et promotion')
plt.legend(title='Promotions')
plt.show()

Correlation

In [ ]:
corr = df[num_cols].corr()
sns.heatmap(corr, annot=False, cmap='plasma')
plt.show()

Regression Lineaire

In [4]:
X = df.drop(['Ventes'], axis=1)
y = df['Ventes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42069)

In [ ]:
reg = LinearRegression()
#train model`
reg.fit(X_train,y_train)
print(reg.score(X_test,y_test))
df_predict = pd.DataFrame(reg.predict(X_test), columns=['Predicted'])
df_predict['Actual'] = y_test.values
df_predict.head(10)

Regression Logistique

In [ ]:
df.drop(['Ventes'], axis=1, inplace=True)
X = df.drop(['cat_Ventes'], axis=1)
y = df['cat_Ventes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42069)

In [9]:
model_logistic = LogisticRegression(solver='lbfgs',random_state=0).fit(X, y)
score = cross_val_score(model_logistic, X, y, cv=10, scoring='accuracy')
print(f"score : {score}")
print(f"score moyen : {np.mean(score)}")
#print(f"score : {model_logistic.score(df[selected_columns], df['TypeDeMagasin'])}")
#print(f"prediction : { model_logistic.predict(df[selected_columns])}")
#print(f"predict_probat : {model_logistic.predict_proba(df[selected_columns])}")


score : [0.50741529 0.47316005 0.52147504 0.52221503 0.59094133 0.54854623
 0.51162432 0.57193513 0.52482116 0.58812284]
score moyen : 0.5360256421035778


In [ ]:
#confusion_matrix = pd.crosstab(df['TypeDeMagasin'], model_logistic.predict(df['TypeDeMagasin']), rownames=['Classe réelle'], colnames=['Classe prédite'])
display = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y,model_logistic.predict(y)))
display = display.plot(cmap='Blues')
display.show()


0.24588112114867644


,Predicted,Actual
0,4054.671138,5626
1,4490.204155,4210
2,7487.264893,6679
3,3951.676458,4803
4,3976.847668,3760
5,4413.908236,5687
6,3947.115952,0
7,3912.617125,3732
8,4415.288189,4870
9,7619.784685,12403


In [77]:
cross_val = cross_val_score(reg, X, y, cv=5)
print(cross_val)


[0.26671742 0.23120553 0.22138265 0.24246607 0.23410037]
